In [ ]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16
import glob
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.models import model_from_json
import keras
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path_structure = '/content/gdrive/My Drive/autobot/custom/tl_model_structure.json'
path_weights = '/content/gdrive/My Drive/autobot/custom/tl_model_weights.h5'

path_animal = '/content/gdrive/My Drive/autobot/custom/animal_data/'
path_person = '/content/gdrive/My Drive/autobot/custom/person_data/'
path_stop = '/content/gdrive/My Drive/autobot/custom/stop_data/'
path_cone = '/content/gdrive/My Drive/autobot/custom/cone_data/'
path_lane = '/content/gdrive/My Drive/autobot/custom/lane_data/'

path_red = '/content/gdrive/My Drive/autobot/custom/red_data/'
path_green = '/content/gdrive/My Drive/autobot/custom/green_data/'
path_blue = '/content/gdrive/My Drive/autobot/custom/blue_data/'

path_image = '/content/gdrive/My Drive/autobot/custom/green_data/0bed40e4-b01b-11ec-b170-5ca6e6e3ecc8.jpg'

path_x_train_dat = '/content/gdrive/My Drive/autobot/custom/x_train.dat'
path_y_train_dat = '/content/gdrive/My Drive/autobot/custom/y_train.dat'
path_x_test_dat = '/content/gdrive/My Drive/autobot/custom/x_test.dat'
path_y_test_dat = '/content/gdrive/My Drive/autobot/custom/y_test.dat'

In [ ]:
images = []
labels = []

In [ ]:
paths = [path_animal,path_person,path_stop,path_cone,path_red,path_green,path_blue]
for i in range(len(paths)):
  for img in glob.glob(paths[i] + "*.jpg"):
    try:
      img = image.load_img(img)
      #img = Image.open(img)
      img = img.resize((112,112))
      image_array = image.img_to_array(img)
      images.append(image_array)
      labels.append(i)

    except:
      pass
    

In [ ]:
print(len(images))
print(len(labels))

6202
6202


In [ ]:
x = np.array(images)
y = np.array(labels)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [ ]:
x_train = vgg16.preprocess_input(x_train)
x_test = vgg16.preprocess_input(x_test)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(5271, 112, 112, 3)
(5271,)
(931, 112, 112, 3)
(931,)


In [ ]:
y_train = keras.utils.np_utils.to_categorical(y_train, 7)
y_test = keras.utils.np_utils.to_categorical(y_test, 7)

In [ ]:
print(y_train.shape)
print(y_test.shape)

(5271, 7)
(931, 7)


In [ ]:
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

In [ ]:
features_x_train = pretrained_nn.predict(x_train)
features_x_test = pretrained_nn.predict(x_test)

In [ ]:
print(features_x_train.shape)

(5271, 3, 3, 512)


In [ ]:
joblib.dump(features_x_train, path_x_train_dat)
joblib.dump(y_train, path_y_train_dat)
joblib.dump(features_x_test, path_x_test_dat)
joblib.dump(y_test, path_y_test_dat)

['/content/gdrive/My Drive/autobot/custom/y_test.dat']

In [ ]:
x_train = joblib.load(path_x_train_dat)
y_train = joblib.load(path_y_train_dat)
x_test = joblib.load(path_x_test_dat)
y_test = joblib.load(path_y_test_dat)

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='sigmoid'))

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data=(x_test, y_test),
    shuffle=True
)

Epoch 1/20
165/165 [==============================] - 3s 11ms/step - loss: 0.2117 - accuracy: 0.9632 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 2/20
165/165 [==============================] - 2s 14ms/step - loss: 0.0219 - accuracy: 0.9983 - val_loss: 3.2436e-05 - val_accuracy: 1.0000
Epoch 3/20
165/165 [==============================] - 3s 18ms/step - loss: 0.0104 - accuracy: 0.9991 - val_loss: 6.6749e-07 - val_accuracy: 1.0000
Epoch 4/20
165/165 [==============================] - 2s 10ms/step - loss: 0.0188 - accuracy: 0.9985 - val_loss: 5.3569e-06 - val_accuracy: 1.0000
Epoch 5/20
165/165 [==============================] - 2s 10ms/step - loss: 0.0085 - accuracy: 0.9994 - val_loss: 3.4439e-07 - val_accuracy: 1.0000
Epoch 6/20
165/165 [==============================] - 2s 11ms/step - loss: 0.0108 - accuracy: 0.9992 - val_loss: 2.7271e-04 - val_accuracy: 1.0000
Epoch 7/20
165/165 [==============================] - 2s 10ms/step - loss: 0.0083 - accuracy: 0.9998 - val_loss: 6.2176e-0

In [ ]:
model_structure = model.to_json()
f = Path(path_structure)
f.write_text(model_structure)
model.save_weights(path_weights)

In [ ]:
f1 = Path(path_structure)
model_structure = f.read_text()
model = model_from_json(model_structure)
model.load_weights(path_weights)

In [ ]:
img = image.load_img(path_image, target_size=(112, 112))
image_array = image.img_to_array(img)
images = np.expand_dims(image_array, axis=0)
images = vgg16.preprocess_input(images)

In [ ]:
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(112, 112, 3))
features = feature_extraction_model.predict(images)

In [ ]:
results = model.predict(features)
single_result = results[0][0]
print(results)

[[0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
a = [1,2,3,4,5]

In [ ]:
b = a[:3]
print(b)
c = a[3:]
print(c)

[1, 2, 3]
[4, 5]
